In [68]:
import torch
import torch.nn as nn
import sys
import nltk
import torch.nn.functional as F
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
sys.path.insert(0, '..')
import pickle
from torch.autograd import grad as torch_grad

word_embedding_dim = 50
z_size = 100
output_shape = (1, 105, 8)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [69]:
print(torch.__version__)
print("GPU Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = "cpu"

2.0.1
GPU Available: False


In [70]:
import pickle

# To load the lists from the file:
with open(r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs.pkl", 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [34]:
def create_word_label_embeddings(Word_Labels_List):
    tokenized_words = []
    for i in range(len(Word_Labels_List)):
        tokenized_words.append([Word_Labels_List[i]])
    model = Word2Vec(sentences=tokenized_words, vector_size=word_embedding_dim, window=5, min_count=1, workers=4)
    word_embeddings = {word: model.wv[word] for word in model.wv.index_to_key}
    print("Number of word embeddings:", len(word_embeddings))
    #word, embedding = list(word_embeddings.items())[10]
    #print(f"Word: {word}, Embedding: {embedding}")

    Embedded_Word_labels = []
    for word in EEG_word_level_labels:
        Embedded_Word_labels.append(word_embeddings[word])

    return Embedded_Word_labels, word_embeddings

def create_dataloader(EEG_word_level_embeddings, Embedded_Word_labels):
    #EEG_word_level_embeddings_normalize = (EEG_word_level_embeddings - np.mean(EEG_word_level_embeddings)) / np.std(EEG_word_level_embeddings)

    # Assuming EEG_synthetic is the generated synthetic EEG data
    #EEG_synthetic_denormalized = (EEG_synthetic * np.max(np.abs(EEG_word_level_embeddings))) + np.mean(EEG_word_level_embeddings)


    EEG_word_level_embeddings_normalize = (EEG_word_level_embeddings - np.mean(EEG_word_level_embeddings)) / np.max(np.abs(EEG_word_level_embeddings))


    float_tensor = torch.tensor(EEG_word_level_embeddings_normalize, dtype=torch.float)
    float_tensor = float_tensor.unsqueeze(1)

    #print(EEG_word_level_embeddings_normalize)
    # Calculate mean and standard deviation
    print(torch.isnan(float_tensor).any())

    train_data = []
    for i in range(len(float_tensor)):
       train_data.append([float_tensor[i], Embedded_Word_labels[i]])
    trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)
    return trainloader

In [21]:
Embedded_Word_labels, word_embeddings = create_word_label_embeddings(EEG_word_level_labels)
#trainloader = create_dataloader(EEG_word_level_embeddings, Embedded_Word_labels)

Number of word embeddings: 2397


In [43]:
Embedded_Word_labels

[array([-0.00994713, -0.00256661,  0.00656127, -0.01282807, -0.01940318,
        -0.01852047,  0.01804139,  0.01074339, -0.00957645, -0.01665928,
         0.0025879 ,  0.00575613, -0.00249057,  0.00254174, -0.00864261,
         0.00958273,  0.00295037,  0.01775565, -0.01995303, -0.01053914,
        -0.01820568, -0.00069584, -0.01571461,  0.01006248, -0.01279371,
        -0.01190567,  0.01014182, -0.01631954,  0.0029104 , -0.01447908,
         0.01972484,  0.01726752,  0.00353791,  0.01157701,  0.00919243,
        -0.01198357,  0.01951389, -0.01936441,  0.01609852,  0.00551276,
        -0.00611024, -0.00712373,  0.01814391, -0.01088182,  0.01637374,
        -0.01201777,  0.01678275, -0.00111099,  0.0158852 , -0.00630994],
       dtype=float32),
 array([ 0.00273734, -0.01224787, -0.01464311, -0.00432493, -0.00950043,
         0.01976222,  0.00867577, -0.00135459, -0.00083622, -0.00978772,
        -0.01588598,  0.01866005, -0.01965163,  0.01572459, -0.00969919,
         0.00988495, -0.015

In [22]:
word_embedding = word_embeddings['1965']

In [23]:
def create_noise(batch_size, z_size, mode_z):
    if mode_z == 'uniform':
        input_z = torch.rand(batch_size, z_size)*2 - 1
    elif mode_z == 'normal':
        input_z = torch.randn(batch_size, z_size)
    return input_z

In [24]:
input_z = create_noise(1, 100, "uniform")

In [25]:
def generate_samples(g_model, input_z, input_t):
    # Create random noise as input to the generator
    # Generate samples using the generator model
    with torch.no_grad():
        g_output = g_model(input_z, input_t)

    return g_output.cpu().numpy()

In [30]:
class Generator(nn.Module):
    def __init__(self, noise_dim, word_embedding_dim):
        super(Generator, self).__init__()

        self.noise_dim = noise_dim
        self.word_embedding_dim = word_embedding_dim

        # Define the layers of your generator
        self.fc_noise = nn.Linear(noise_dim, 105*8)  # Increase the size for more complexity
        self.fc_word_embedding = nn.Linear(word_embedding_dim, 105*8)  # Increase the size for more complexity
        self.conv1 = nn.Conv2d(2, 128, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(128)
        self.relu = nn.LeakyReLU(0.2)

        self.conv2 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, noise, word_embedding):
        # Process noise
        noise = self.fc_noise(noise)
        noise = noise.view(noise.size(0), 1, 105,8)  # Adjust the size to match conv1

        # Process word embedding
        word_embedding = self.fc_word_embedding(word_embedding.to(device))
        word_embedding = word_embedding.view(word_embedding.size(0), 1, 105, 8)  # Adjust the size to match conv1

        # Concatenate noise and word embedding
        combined_input = torch.cat([noise, word_embedding], dim=1)

        # Upsample and generate the output
        z = self.conv1(combined_input)
        z = self.bn1(z)
        z = self.relu(z)

        z = self.conv2(z)
        z = self.bn2(z)
        z = self.relu(z)

        z = self.conv3(z)
        z = self.tanh(z)

        return z

In [31]:
gen_model = Generator(z_size, word_embedding_dim)  # Replace with your actual generator model class
checkpoint = torch.load(r"I:\Science\CIS-YASHMOSH\niallmcguire\WGAN_Text_2.0\Textual_WGAN_GP_checkpoint_epoch_100.pt",
                        map_location=torch.device('cpu'))

In [32]:
# Load the model's state_dict onto the CPU
gen_model.load_state_dict(checkpoint['gen_model_state_dict'])

# Set the model to evaluation mode
gen_model.eval()

Generator(
  (fc_noise): Linear(in_features=100, out_features=840, bias=True)
  (fc_word_embedding): Linear(in_features=50, out_features=840, bias=True)
  (conv1): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): LeakyReLU(negative_slope=0.2)
  (conv2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (tanh): Tanh()
)

In [44]:
word_embedding_tensor = torch.tensor(word_embedding, dtype=torch.float)

In [48]:
word_embedding_tensor = word_embedding_tensor.unsqueeze(0)
word_embedding_tensor

tensor([[[-0.0102, -0.0068,  0.0109,  0.0090, -0.0137,  0.0110,  0.0009,
          -0.0119, -0.0096,  0.0157,  0.0151, -0.0137,  0.0185,  0.0147,
           0.0158,  0.0026, -0.0005,  0.0002, -0.0170, -0.0195, -0.0020,
          -0.0174, -0.0028,  0.0041,  0.0113,  0.0096, -0.0153,  0.0067,
           0.0124,  0.0128,  0.0166,  0.0075,  0.0025, -0.0090,  0.0040,
           0.0116, -0.0045,  0.0140, -0.0022, -0.0105, -0.0056,  0.0195,
           0.0018, -0.0185, -0.0031, -0.0081, -0.0098,  0.0192,  0.0106,
          -0.0123]]])

In [53]:
g_output = generate_samples(gen_model, input_z, word_embedding_tensor)
EEG_synthetic_denormalized = (g_output * np.max(np.abs(EEG_word_level_embeddings))) + np.mean(EEG_word_level_embeddings)

In [60]:
synthetic_sample = torch.tensor(EEG_synthetic_denormalized[0][0], dtype=torch.float)

In [67]:
synthetic_sample.resize(840)

tensor([ 4.6282e-01,  7.3410e-01,  1.0351e+00,  9.8283e-01,  4.7440e-01,
         8.8473e-01,  8.9377e-01,  1.4107e+00,  8.9564e-01,  1.1317e+00,
         5.4706e-01,  1.4678e+00,  1.4822e+00,  1.6691e+00,  1.4442e+00,
         9.3821e-01,  2.5203e-01,  1.4391e+00,  9.1379e-01,  1.6827e+00,
         1.1325e+00,  6.2076e-01,  9.0900e-01,  4.5878e-01,  1.2563e+00,
         1.4610e+00,  9.7711e-01,  1.1579e+00,  1.9283e+00,  6.7519e-01,
         9.0437e-01,  1.5350e+00,  1.4240e+00,  7.1623e-01,  9.0562e-01,
         1.4038e+00,  2.8811e+00,  1.4201e+00,  1.1474e+00,  1.1710e+00,
         1.8994e+00,  1.8636e+00,  1.5089e+00,  9.2435e-01,  9.9575e-01,
         1.3410e+00,  3.9917e+00,  1.6816e+00,  2.3014e+00,  1.7574e+00,
         1.3880e+00,  1.6773e+00,  4.5200e+00,  3.5631e+00,  1.9619e+00,
         1.7179e+00,  3.9829e+00,  3.3282e+00,  8.7335e-01,  8.7481e-01,
         3.3858e+00,  3.2227e+00,  2.4590e+00,  1.3643e+00,  1.1765e+00,
         1.6890e+00,  1.4778e+00,  4.2052e+00,  3.1